### 🎬 Análise de Sentimento de Críticas de Filmes

#### **Objetivo do Projeto**

O principal objetivo deste projeto é construir um modelo de **Análise de Sentimento** capaz de classificar automaticamente críticas de filmes do IMDb como **positivas** ou **negativas**. Para alcançar isso, iremos:

1.  **Explorar e Pré-processar** um grande conjunto de dados de avaliações, tratando o texto para remover informações irrelevantes e extrair características úteis.
2.  **Vetorizar** as palavras, transformando-as em um formato numérico que os modelos de machine learning podem entender.
3.  **Treinar e Otimizar** um modelo de **Regressão Logística**, avaliando seu desempenho antes e depois do pré-processamento.
4.  **Analisar** as palavras mais influentes na classificação, entendendo o que o modelo "aprendeu".
5.  **Construir** um pipeline completo que possa ser usado para prever o sentimento de novas avaliações de filmes.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('IMDB Dataset.csv')

In [3]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.shape

(50000, 2)

In [7]:
df.value_counts("sentiment")

sentiment
negative    25000
positive    25000
Name: count, dtype: int64

## 📄 Vetorização do Texto com TF-IDF

Nesta etapa, o objetivo é transformar o texto das avaliações em um formato numérico que os modelos de machine learning possam entender. A técnica escolhida para isso é a **TF-IDF (Term Frequency-Inverse Document Frequency)**, que calcula a importância de cada palavra em um documento em relação a todos os outros.

O `TfidfVectorizer` foi configurado para:
- Não converter o texto para letras minúsculas (`lowercase=False`), pois o tratamento de texto será feito em uma etapa posterior.
- Considerar até **10.000** das palavras e combinações de palavras (N-grams) mais relevantes.
- Usar N-grams de **1 a 2 palavras** (`ngram_range=(1,2)`), o que nos permite capturar o contexto de combinações de palavras, como "very good" ou "not bad".

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(lowercase=False, max_features=10000, ngram_range=(1,2))
palavras_vetorizadas = tfidf.fit_transform(df.review)

### 📈 Separação dos Dados para Treino e Teste

Para avaliar o desempenho do nosso modelo de forma justa, é crucial separar os dados em conjuntos de treino e teste. O conjunto de treino será usado para que o modelo "aprenda" a relação entre as palavras e o sentimento (positivo ou negativo), enquanto o conjunto de teste será usado para verificar quão bem o modelo generaliza para dados que ele nunca viu antes.

In [74]:
from sklearn.model_selection import train_test_split

X_treino, X_teste, y_treino, y_teste = train_test_split(palavras_vetorizadas, df.sentiment, random_state=2905)

## 🤖 Treinamento do Modelo de Regressão Logística

Agora que nossos dados estão prontos, podemos treinar o nosso primeiro modelo de classificação. Escolhemos a **Regressão Logística**, que é um algoritmo robusto e eficiente, ideal para problemas de classificação binária como este (positivo vs. negativo).

O modelo é inicializado com `solver='saga'` e `max_iter=5000` para garantir a convergência do algoritmo, ou seja, que ele encontre a melhor solução para os dados.

Após o treinamento com os dados de treino (`X_treino`, `y_treino`), calculamos a acurácia do modelo no conjunto de teste (`X_teste`, `y_teste`).

In [75]:
from sklearn.linear_model import LogisticRegression

modelo_regressao_logistica = LogisticRegression(solver='saga', max_iter=5000)
modelo_regressao_logistica.fit(X_treino, y_treino)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'saga'
,max_iter,5000
,multi_class,'deprecated'


In [78]:
acuracia_modelo_um = modelo_regressao_logistica.score(X_teste, y_teste) *100
print(f"Acuracia do modelo: {acuracia_modelo_um:.2f}%")

Acuracia do modelo: 89.86%


## 🧼 Análise e Pré-processamento do Texto

Antes de seguir com a modelagem, vamos realizar uma análise detalhada do texto para identificar e tratar elementos que podem melhorar o desempenho do nosso modelo. A primeira etapa é a análise de **frequência de palavras**.

In [23]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
todas_reviews = [review for review in df.review]
texto_unico = ' '.join(todas_reviews)

In [33]:
from nltk import tokenize

token_espaco = tokenize.WhitespaceTokenizer()
token_frase = token_espaco.tokenize(texto_unico)

In [34]:
frequencia = nltk.FreqDist(token_frase)
df_frequencia = pd.DataFrame({
    "Palavra": list(frequencia.keys()),
    "Frequencia": list(frequencia.values())
})
df_frequencia.head()

,Palavra,Frequencia
0,One,3460
1,of,283625
2,the,568735
3,other,15745
4,reviewers,413


In [35]:
df_frequencia.nlargest(columns='Frequencia',n=10)

,Palavra,Frequencia
2,the,568735
52,a,306960
38,and,301919
1,of,283625
64,to,261850
22,is,203056
44,in,169981
151,I,132498
7,that,126818
21,this,113726


### A Importância de Ignorar as Palavras Comuns

As palavras mais frequentes, como **"the"**, **"a"**, **"and"** e **"of"**, não carregam significado emocional e, por isso, não ajudam a explicar o sentimento de uma avaliação. Elas são consideradas **"stopwords"** e precisam ser removidas no pré-processamento do texto para que o modelo possa focar em termos mais relevantes, como adjetivos e substantivos, que realmente indicam se uma crítica é positiva ou negativa.

In [36]:
stopwords = nltk.corpus.stopwords.words('english')

In [37]:
stopwords

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

## ✂️ Melhorando o Pré-processamento: Stemming

Nesta seção, vamos aprimorar o pré-processamento do texto. Além de remover as *stopwords*, vamos aplicar o **Stemming**, uma técnica que reduz as palavras à sua forma radical. Por exemplo, "loving," "loved," e "loves" são todos reduzidos à raiz "love."

Isso ajuda o modelo a tratar palavras com significados semelhantes de forma unificada, reduzindo a dimensionalidade dos dados e, potencialmente, melhorando a performance.

In [55]:
from nltk.stem import PorterStemmer

# Cria um tokenizador que separa o texto em palavras (ignora pontuação)
tokenizador = tokenize.RegexpTokenizer(r'\w+')

# Cria um stemmer usando o algoritmo de Porter, que reduz palavras à sua raiz
stemmer = PorterStemmer()

In [79]:
def preprocessa(texto):
    texto = texto.lower() # lowercase

    palavras = tokenizador.tokenize(texto) # tokenização

    palavras = [p for p in palavras if p not in stopwords] # remove stopwords

    palavras = [stemmer.stem(p) for p in palavras] # pega só o radical da palavra

    return ' '.join(palavras)

In [80]:
df['tratamento'] = df['review'].apply(preprocessa)

In [81]:
df.head()

,review,sentiment,tratamento
0,One of the other reviewers has mentioned that ...,positive,one review mention watch 1 oz episod hook righ...
1,A wonderful little production. <br /><br />The...,positive,wonder littl product br br film techniqu unass...
2,I thought this was a wonderful way to spend ti...,positive,thought wonder way spend time hot summer weeke...
3,Basically there's a family where a little boy ...,negative,basic famili littl boy jake think zombi closet...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter mattei love time money visual stun film...


In [85]:
tfidf = TfidfVectorizer(lowercase=False, max_features=10000, ngram_range=(1,2))
palavras_vetorizadas = tfidf.fit_transform(df.tratamento)

X_treino, X_teste, y_treino, y_teste = train_test_split(palavras_vetorizadas, df['sentiment'], random_state=2905)

modelo_regressao_logistica = LogisticRegression(solver='saga', max_iter=5000)
modelo_regressao_logistica.fit(X_treino, y_treino)

acuracia_apos_processamento = modelo_regressao_logistica.score(X_teste, y_teste) * 100
print(f"Acuracia do modelo após processamento: {acuracia_apos_processamento:.2f}%")

Acuracia do modelo após processamento: 89.74%


## 🎯 Reavaliando o Modelo com um Pré-processamento Ajustado

Depois de experimentar com o stemming, percebemos que a acurácia do modelo não melhorou como esperávamos. A próxima tentativa é focar em um pré-processamento mais simples, que apenas converte o texto para minúsculas e remove as *stopwords*.

A ausência do stemming permite que o modelo capture nuances de palavras que o algoritmo de redução de radical poderia ter "simplificado" demais.

In [88]:
def preprocessa_ajustado(texto):
    texto = texto.lower() # lowercase

    palavras = tokenizador.tokenize(texto) # tokenização

    palavras = [p for p in palavras if p not in stopwords] # remove stopwords

    return ' '.join(palavras)

In [91]:
df['tratamento_ajustado'] = df['review'].apply(preprocessa_ajustado)

In [92]:
tfidf = TfidfVectorizer(lowercase=False, max_features=10000, ngram_range=(1,2))
palavras_vetorizadas = tfidf.fit_transform(df.tratamento_ajustado)

X_treino, X_teste, y_treino, y_teste = train_test_split(palavras_vetorizadas, df['sentiment'], random_state=2905)

modelo_regressao_logistica = LogisticRegression(solver='saga', max_iter=5000)
modelo_regressao_logistica.fit(X_treino, y_treino)

acuracia_processamento_ajustado = modelo_regressao_logistica.score(X_teste, y_teste) * 100
print(f"Acuracia do modelo após processamento: {acuracia_processamento_ajustado:.2f}%")

Acuracia do modelo após processamento: 89.97%


In [93]:
pesos = pd.DataFrame(
    modelo_regressao_logistica.coef_[0].T,
    index=tfidf.get_feature_names_out()
)

In [94]:
pesos.nlargest(50, 0)

,0
great,7.087517
excellent,6.657503
perfect,5.326769
amazing,5.110491
best,4.897764
wonderful,4.735690
brilliant,4.453409
loved,4.431045
one best,4.320022
hilarious,4.238733


In [95]:
pesos.nsmallest(50, 0)

,0
worst,-9.119595
bad,-7.374760
awful,-7.293645
waste,-6.805452
boring,-6.432110
terrible,-6.166793
poor,-5.912961
nothing,-5.399318
horrible,-5.206796
dull,-4.862556


In [96]:
import joblib

joblib.dump(tfidf, 'tfidf_vectorizer.pkl')
joblib.dump(modelo_regressao_logistica, 'modelo_regressao_logistica.pkl')

['modelo_regressao_logistica.pkl']

## 🔮 Avaliando Novos Reviews

Finalmente, vamos usar o modelo treinado para fazer previsões em novas avaliações. Para isso, precisamos:

1.  **Carregar** o vetorizador TF-IDF e o modelo de regressão logística que foram salvos anteriormente.
2.  **Pré-processar** as novas avaliações usando a mesma função `preprocessa_ajustado` que o modelo foi treinado.
3.  **Vetorizar** as avaliações pré-processadas com o vetorizador TF-IDF carregado.
4.  **Fazer as previsões** usando o modelo.

O resultado será um DataFrame que compara a avaliação original com a previsão de sentimento (positivo ou negativo) do nosso modelo.

In [97]:
tfidf = joblib.load('tfidf_vectorizer.pkl')
regressao_logistica = joblib.load('modelo_regressao_logistica.pkl')

tokenizador = tokenize.RegexpTokenizer(r'\w+')

In [98]:
novos_reviews = ['I absolutely loved this movie! The story was engaging, the acting was superb, and the cinematography was stunning. I would definitely watch it again.',
                 'This film was a huge disappointment. The plot was predictable, the characters were dull, and it felt way too long. I wouldn’t recommend it to anyone.']

In [99]:
novos_reviews_processados = [preprocessa_ajustado(review) for review in novos_reviews]

In [100]:
novos_reviews_processados

['absolutely loved movie story engaging acting superb cinematography stunning would definitely watch',
 'film huge disappointment plot predictable characters dull felt way long recommend anyone']

In [102]:
novos_reviews_tfidf = tfidf.transform(novos_reviews_processados)

predicoes = regressao_logistica.predict(novos_reviews_tfidf)

df_previsoes = pd.DataFrame({
    'Avaliacao': novos_reviews,
    'Sentimento_Previsto': predicoes
})

df_previsoes

,Avaliacao,Sentimento_Previsto
0,I absolutely loved this movie! The story was e...,positive
1,This film was a huge disappointment. The plot ...,negative
